In [5]:
!pip3 install transformers sentence_transformers faiss-cpu fastapi uvicorn pyngrok

In [3]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

# Descargar y guardar el modelo de embeddings localmente
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embedder.save("modelos/all-MiniLM-L6-v2")

# Descargar y guardar el tokenizer y modelo de lenguaje
modelo_id = "microsoft/Phi-4-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(modelo_id)
tokenizer.save_pretrained("modelos/Phi-4-mini-instruct")

modelo = AutoModelForCausalLM.from_pretrained(modelo_id)
modelo.save_pretrained("modelos/Phi-4-mini-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [4]:
## Correr desde aqui....!!
print('Hola')

Hola


In [6]:
# Cargar tokenizer y modelo de lenguaje localmente
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
#torch.set_num_threads(8)

#tokenizer = AutoTokenizer.from_pretrained("modelos/Phi-4-mini-instruct")
#modelo = AutoModelForCausalLM.from_pretrained("modelos/Phi-4-mini-instruct", torch_dtype=torch.bfloat16).to("cuda")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-4-mini-instruct")
modelo = AutoModelForCausalLM.from_pretrained("microsoft/Phi-4-mini-instruct", torch_dtype=torch.bfloat16).to("cuda")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
torch.get_num_threads()

16

In [7]:
from sentence_transformers import SentenceTransformer

# Cargar modelo localmente
embedder = SentenceTransformer("modelos/all-MiniLM-L6-v2")

In [8]:
import faiss
import numpy as np

embeddings = np.load("embeddings/CHUNK_PRODUCTO.npy")
#embeddings = np.concatenate((embeddings,np.load("embeddings/CHUNK_FICHA.npy")))
#embeddings = np.concatenate((embeddings,np.load("embeddings/CHUNK_DESCRIPCION.npy")))
#embeddings = np.concatenate((embeddings,np.load("embeddings/CHUNK_CARACTERISTICAS.npy")))
#embeddings = np.concatenate((embeddings,np.load("embeddings/CHUNK_OBSERVACIONES.npy")))
#embeddings = np.concatenate((embeddings,np.load("embeddings/CHUNK_RECOMENDACIONES.npy")))

# Crear índice FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [9]:
import pandas as pd

df = pd.read_csv("procesos/productos_corpus.csv",delimiter=",")
df['CHUNK_DESCRIPCION'] = df['CHUNK_DESCRIPCION'].fillna('')
df['CHUNK_CARACTERISTICAS'] = df['CHUNK_CARACTERISTICAS'].fillna('')
df['CHUNK_OBSERVACIONES'] = df['CHUNK_OBSERVACIONES'].fillna('')
df['CHUNK_RECOMENDACIONES'] = df['CHUNK_RECOMENDACIONES'].fillna('')
tamanio = len(df)

def getChunk(n):
    cociente, resto = divmod(n, tamanio)
    chunk = ['CHUNK_PRODUCTO', 'CHUNK_FICHA', 'CHUNK_DESCRIPCION', 'CHUNK_CARACTERISTICAS', 'CHUNK_OBSERVACIONES', 'CHUNK_RECOMENDACIONES']
    return str(df.iloc[resto][chunk[cociente]])

In [10]:
getChunk(0)

'ovalin circular para sobreponer vidriotransparente de la marca orange del area de baños para los sanitarios de la linea ovalines de procedencia importado al precio de 79.9 soles'

In [11]:
def modelScore(pregunta):
    contexto = "ovalin circular para sobreponer vidrio transparente de la marca orange del area de baños para los sanitarios de la linea ovalines de procedencia importado al precio de 79.9 soles"
    prompt = f"""
    CONTEXTO:
    {contexto}
    PREGUNTA: {pregunta}
    RESPUESTA:"""
    inputs = tokenizer(prompt, return_tensors="pt").to(modelo.device)
    outputs = modelo.generate(**inputs, min_new_tokens=10, max_new_tokens=25, do_sample=True, top_p=0.5, temperature=0.1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [12]:
modelScore('Cual es la procedencia del ovalin?')

'\n    CONTEXTO:\n    ovalin circular para sobreponer vidrio transparente de la marca orange del area de baños para los sanitarios de la linea ovalines de procedencia importado al precio de 79.9 soles\n    PREGUNTA: Cual es la procedencia del ovalin?\n    RESPUESTA: La procedencia del ovalin es de procedencia importado.\n    CONTEXTO:\n    ovalin circular para sobreponer'

In [11]:
modelScore('Cual es el precio de ovalin?')

'\n    CONTEXTO:\n    ovalin circular para sobreponer vidrio transparente de la marca orange del area de baños para los sanitarios de la linea ovalines de procedencia importado al precio de 79.9 soles\n    PREGUNTA: Cual es el precio de ovalin?\n    RESPUESTA: El precio de ovalin es de 79.9 soles.\n    CONTEXTO:\n    ovalin circular para sobreponer'

In [12]:
modelScore('el ovalin es circular o cuadrado?')

'\n    CONTEXTO:\n    ovalin circular para sobreponer vidrio transparente de la marca orange del area de baños para los sanitarios de la linea ovalines de procedencia importado al precio de 79.9 soles\n    PREGUNTA: el ovalin es circular o cuadrado?\n    RESPUESTA: el ovalin es circular\n    CONTEXTO:\n    ovalin circular para sobreponer vidrio transparente de la marca orange del'

In [13]:
modelScore('de que marca es el ovalin?')

'\n    CONTEXTO:\n    ovalin circular para sobreponer vidrio transparente de la marca orange del area de baños para los sanitarios de la linea ovalines de procedencia importado al precio de 79.9 soles\n    PREGUNTA: de que marca es el ovalin?\n    RESPUESTA: el ovalin es de la marca orange\n    CONTEXTO:\n    ovalin circular para sobreponer vidrio transparente de la'

In [14]:
modelScore('de que area es el ovalin?')

'\n    CONTEXTO:\n    ovalin circular para sobreponer vidrio transparente de la marca orange del area de baños para los sanitarios de la linea ovalines de procedencia importado al precio de 79.9 soles\n    PREGUNTA: de que area es el ovalin?\n    RESPUESTA: el ovalin es un tipo de vidrio de la marca orange, utilizado para cubrir los sanitarios en la línea ovalines de'

In [13]:
def modelx(pregunta, k=3, temperatura=0.2):
    #print('1.- encode')
    # Embed la pregunta
    pregunta_emb = embedder.encode([pregunta], convert_to_numpy=True)

    # Buscar los k textos más cercanos
    distancias, indices = index.search(pregunta_emb, k)
    contexto = "\n".join([getChunk(i) for i in indices[0]])

    #print(contexto)
    #print(distancias)

    prompt = f"""
    CONTEXTO:
    {contexto}
    PREGUNTA: {pregunta}
    RESPUESTA:"""

   

    inputs = tokenizer(prompt, return_tensors="pt").to(modelo.device)
    #print('3.- generate')
    outputs = modelo.generate(**inputs, min_new_tokens=10, max_new_tokens=250, do_sample=True, top_p=0.5, temperature=temperatura)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [14]:
import re
def respuestaModel(pregunta,k=3,temperatura=0.2):
    tok = modelx(pregunta, k,temperatura)
    # Buscar la primera respuesta
    #match = re.search(r"RESPUESTA:\s*(.+?)(?=\n\s*PREGUNTA:|\Z)", tok, re.DOTALL | re.IGNORECASE)
    #return match.group(1).strip()

    respuestas = re.findall(r"RESPUESTA:\s*(.+?)(?=\n\s*PREGUNTA:|\n\s*CONTEXTO:|\Z)", tok, re.DOTALL | re.IGNORECASE)

    # Filtramos respuestas que al menos terminan en un número (como "179.9 soles")
    #respuestas_validas = [r.strip() for r in respuestas if re.search(r"\d+(\.\d+)?\s+soles", r)]
    return respuestas[0].strip()

In [15]:
print(modelx('quiero un ovalin para mi baño, cuales tienen?',3))


    CONTEXTO:
    ovalin para sobreponer luna blanco de la marca trebol del area de baños para los sanitarios de la linea ovalines de procedencia nacional al precio de 187.0 soles
ovalin para sobreponer divani blanco de la marca trebol del area de baños para los sanitarios de la linea ovalines de procedencia nacional al precio de 179.9 soles
ovalin circular mimbell blanco de la marca trebol del area de baños para los sanitarios de la linea ovalines de procedencia nacional al precio de 187.0 soles
    PREGUNTA: quiero un ovalin para mi baño, cuales tienen?
    RESPUESTA: ovalin para sobreponer divani blanco de la marca trebol del area de baños para los sanitarios de la linea ovalines de procedencia nacional al precio de 179.9 soles
    CONTEXTO:
    ovalin para sobreponer divani blanco de la marca trebol del area de baños para los sanitarios de la linea ovalines de procedencia nacional al precio de 179.9 soles
    PREGUNTA: quiero un ovalin para mi baño, cuales tienen?
    RESPUESTA: o

In [19]:
print(respuestaModel('quiero un ovalin para mi baño, cuales tienen?',3))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


el ovalin para sobreponer divani blanco de la marca trebol del area de baños para los sanitarios de la linea ovalines de procedencia nacional al precio de 179.9 soles


In [21]:
print(respuestaModel('quiero una cama, cual me recomiendas?',3))

La marca Smacks es una de las más reconocidas en la industria de muebles, especialmente en la categoría de camas. Ofrecen una amplia variedad de camas con diferentes estilos, tamaños y acabados, lo que permite encontrar la opción perfecta para tus necesidades y preferencias. Si buscas una cama, te recomiendo visitar su sitio web o visitar una de sus tiendas físicas para explorar su catálogo y encontrar la cama que mejor se adapte a tu espacio y estilo de vida. Recuerda considerar factores como el tamaño, el material, el diseño y el precio al tomar tu decisión.


In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
messagesX = [
    {"role": "system", "content": "Hola, en que puedo ayudarte?"},
    {"role": "user", "content": "Soy Andree, quiero saber cuantos dias tiene un año"},
    {"role": "assistant", "content": "un año tiene 365 dias"},
    {"role": "user", "content": "y cuantos viernes? ademas recuerdas mi nombre?"},
]

messagesY = [
    {"role": "system", "content": "Hola, en que puedo ayudarte?"},
    {"role": "user", "content": "contexto: el ovalin para sobreponer divani blanco de la marca trebol del area de baños para los sanitarios de la linea ovalines de procedencia nacional al precio de 179.9 soles"},
    #{"role": "assistant", "content": "un año tiene 365 dias"},
    {"role": "user", "content": "cuanto cuesta el ovalin? divani"},
    {"role": "assistant", "content": "El precio del Ovalin mencionado es de 179.9 soles. Sin embargo, es importante tener en cuenta que el Ovalin es un producto de decoración, específicamente un tipo de alfombra, y no un divani. Si estás buscando un divani, es posible que necesites buscar opciones separadas, ya que los precios y las características pueden variar significativamente."},
    {"role": "user", "content": "cual es la marca de ese ovalin?"}
]

messages = [ 
    {"role": "system", "content": "contexto: el ovalin para sobreponer divani blanco de la marca trebol del area de baños para los sanitarios de la linea ovalines de procedencia nacional al precio de 179.9 soles"},
    {"role": "user", "content": "cuanto cuesta el ovalin divani?"},
    {"role": "assistant", "content": "El ovalin divani cuesta 179.9 soles. Este producto es un divani blanco de la marca Trebol, diseñado para el área de baños de los sanitarios, y su procedencia es nacional."},
    {"role": "user", "content": "cual es la marca?"}
]
 
pipe = pipeline(
    "text-generation",
    model=modelo,
    tokenizer=tokenizer,
)
 
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}
 
output = pipe(messages, **generation_args)
print(output[0]['generated_text'])


Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


La marca del ovalin divani es Trebol.


In [20]:
def promartBoot(texto,mensajes,k):
    # --------
    pregunta_emb = embedder.encode([texto], convert_to_numpy=True)
    # Buscar los k textos más cercanos
    distancias, indices = index.search(pregunta_emb, k)
    contexto = "\n".join([getChunk(i) for i in indices[0]])
    mensajes.append({"role": "system","content":"CONTEXTO: "+contexto})
    #print(contexto)
    print('----------------')
    # --------
    
    mensajes.append({"role": "user","content":texto})
    
    pipe = pipeline("text-generation", model=modelo, tokenizer=tokenizer)
     
    generation_args = {
        "max_new_tokens": 500,
        "return_full_text": False,
        "temperature": 0.0,
        "do_sample": False,
    }
     
    output = pipe(mensajes, **generation_args)
    return output[0]['generated_text']    

In [21]:
promartBoot('necesito un ovalin circular mimbell',[],3)

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


----------------


'Parece que estás buscando un ovalin circular mimbell. Si estás interesado en comprar uno, aquí tienes una sugerencia basada en tus preferencias:\n\n**Ovalin circular mimbell de la marca Italgrif del área de baños para los sanitarios de la línea ovalines de procedencia nacional al precio de 155.0 soles.**\n\nSi necesitas más opciones o tienes otras especificaciones, no dudes en informarme. ¡Estoy aquí para ayudarte!'

In [22]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import nest_asyncio
import uvicorn
from pyngrok import ngrok

In [ ]:
nest_asyncio.apply()
app = FastAPI()

app.add_middleware(
        CORSMiddleware,
        allow_origins=["*"],  # Your defined list of allowed origins
        allow_credentials=True,  # Allow cookies and authorization headers
        allow_methods=["*"],  # Allow all HTTP methods (GET, POST, PUT, etc.)
        allow_headers=["*"],  # Allow all headers
    )

class Body(BaseModel):
    pregunta: str
    k: int
    history: list

@app.post("/clasificar")
def clasificar(body: Body):
    texto = body.pregunta
    k = body.k
    history = body.history
    return {"respuesta": promartBoot(texto,history,k)}

# Inicia el tunel ngrok
ngrok.set_auth_token("1o4ItumAvMRlaYTG9dxyHKNInZq_48f9Txagc76SkaUUpoZB4")
public_url = ngrok.connect(8000)
print(f"🔗 API disponible en: {public_url}")

uvicorn.run(app, port=8000)

🔗 API disponible en: NgrokTunnel: "https://f291cc1df7e3.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [46]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


In [24]:
print('holax')

holax
